In [ ]:
import torch
from torch.utils.data import DataLoader
from torch import nn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from skimage import io
import skimage
from importlib import reload
from tqdm import tqdm

import folded_dataset
reload(folded_dataset)

In [ ]:
# old code

# root_dir = '/scr/zchen/datasets/CHAMMI'

# dataset_name = 'CP'
# df_path = f'{root_dir}/{dataset_name}/enriched_meta.csv'
# df = pd.read_csv(df_path)

In [ ]:
# make sure to add your own path to the images
#root_dir = '/scr/zchen/datasets/morphem_70k_2.0/'
root_dir = '/scr/nnair/dataset/CHAMMI'
feature_dir = '/scr/nnair/all_features'


# why is dataset name only CP (this variable is not changed anywhere else)
#dataset_name = 'CP'
# dataset_names = ['CP', 'Allen', 'HPA']
# index = 0
# dataset_name = pick_dataset(dataset_names, index)
# df_path = f'{root_dir}{dataset_name}/enriched_meta.csv'
# df = pd.read_csv(df_path)

# def pick_dataset(dataset_names, index):
#     dataset_name = dataset_names[index]
#     index++;
#     return dataset_name

In [ ]:
# dataset = folded_dataset.SingleCellDataset(csv_file=df_path,
#                                            root_dir=root_dir, target_labels='train_test_split')


In [ ]:
# sample_inds = [0, 100, 200]
# for ind in sample_inds:
#     fig, axes = plt.subplots(1,2,figsize=(10,5))
#     axes[0].imshow(io.imread(root_dir + df.iloc[ind].file_path))
#     axes[1].imshow(dataset[ind][0].numpy().transpose(1,2,0)[:,:,:3])


In [ ]:
def configure_dataset(root_dir, dataset_name):
    df_path = f'{root_dir}/{dataset_name}/enriched_meta.csv'
    df = pd.read_csv(df_path)
    dataset = folded_dataset.SingleCellDataset(csv_file=df_path, root_dir=root_dir, target_labels='train_test_split')
    sample_inds = [0, 100, 200]
    for ind in sample_inds:
        fig, axes = plt.subplots(1,2,figsize=(10,5))
        axes[0].imshow(io.imread(root_dir + df.iloc[ind].file_path))
        axes[1].imshow(dataset[ind][0].numpy().transpose(1,2,0)[:,:,:3])

    return dataset

## Convnext Feature Extraction

In [ ]:
from timm import create_model
import torch.nn.functional as F

In [ ]:
def forward(pooling='avg', x: torch.Tensor) -> torch.Tensor:
    x = feature_extractor(x)
    if pooling == 'avg':
        x = F.adaptive_avg_pool2d(x, (1, 1))
    elif pooling == 'max':
        x = F.adaptive_max_pool2d(x, (1, 1))
    elif pooling == 'avg_max':
        x_avg = F.adaptive_avg_pool2d(x, (1, 1))
        x_max = F.adaptive_max_pool2d(x, (1, 1))
        x = torch.cat([x_avg, x_max], dim=1)
    elif pooling == None:
        pass
    else:
        raise ValueError(
            f"Pooling {self.cfg.pooling} not supported. Use one of {FeaturePooling.list()}"
        )
    # x = rearrange(x, "b c h w -> b (c h w)")
    return x

In [ ]:
device = torch.device("cuda:7" if torch.cuda.is_available() else "cpu")

In [ ]:
# I am confused by this because I do not see a directory called "features"
#feature_dir = f'{root_dir}features'
feature_file = 'pretrained_convnext_channel_replicate.npy'
# pooling = 'avg'

In [ ]:
pretrained = True
model = create_model("convnext_tiny.fb_in22k", pretrained=pretrained).to(device)
feature_extractor = nn.Sequential(
                    model.stem,
                    model.stages[0],
                    model.stages[1],
                    model.stages[2].downsample,
                    *[model.stages[2].blocks[i] for i in range(9)],
                    model.stages[3].downsample,
                    *[model.stages[3].blocks[i] for i in range(3)],
                )

In [ ]:
get_save_features(model, feature_dir, feature_file, root_dir)
# train_dataloader = DataLoader(dataset, batch_size=256, shuffle=False)

In [ ]:
# all_feat = []
# for images, label in tqdm(train_dataloader, total=len(train_dataloader)):
#     cloned_images = images.clone()
#     batch_feat = []
#     for i in range(cloned_images.shape[1]):
#         # Copy each channel three times 
#         channel = cloned_images[:, i, :, :]
#         channel = channel.unsqueeze(1)
#         expanded = channel.expand(-1, 3, -1, -1).to(device)

#         # feat_temp = m.forward_features(expanded).cpu().detach().numpy()
#         feat_temp = forward(expanded).cpu().detach().numpy()
#         batch_feat.append(feat_temp)
        
#     batch_feat = np.concatenate(batch_feat, axis=1)
#     all_feat.append(batch_feat)

In [ ]:
# all_feat = np.concatenate(all_feat)
# all_feat.shape

In [ ]:
# all_feat = all_feat.squeeze(2).squeeze(2)
# all_feat.shape

In [ ]:
# feature_path = f'{feature_dir}/{dataset_name}/{feature_file}'
# np.save(feature_path, all_feat)

## ResNet Feature Extraction

In [ ]:
from torchvision.models import resnet18, ResNet18_Weights

In [ ]:
device = torch.device("cuda:5" if torch.cuda.is_available() else "cpu")

In [ ]:
# Same issue regarding "features" directory
# feature_dir = f'{root_dir}features'
feature_file = 'pretrained_resnet18_features.npy'

In [ ]:
weights = ResNet18_Weights.IMAGENET1K_V1
m = resnet18(weights=weights).to(device)
feature_extractor = torch.nn.Sequential(*list(m.children())[:-1]).to(device)

In [ ]:
get_save_features(m, feature_dir, feature_file, root_dir)
# train_dataloader = DataLoader(dataset, batch_size=256, shuffle=False)

In [ ]:
# preprocess = weights.transforms()
# all_feat = []
# for images, label in tqdm(train_dataloader, total=len(train_dataloader)):
#     cloned_images = images.clone()
#     batch_feat = []
#     for i in range(cloned_images.shape[1]):
#         # Copy each channel three times 
#         channel = cloned_images[:, i, :, :]
#         channel = channel.unsqueeze(1)
#         expanded = channel.expand(-1, 3, -1, -1)

#         expanded = preprocess(expanded).to(device)
#         feat_temp = feature_extractor(expanded).cpu().detach().numpy()
#         batch_feat.append(feat_temp)
#     batch_feat = np.concatenate(batch_feat, axis=1)
#     all_feat.append(batch_feat)

In [ ]:
# all_feat = np.concatenate(all_feat)
# all_feat = all_feat.squeeze(2).squeeze(2)
# all_feat.shape

In [ ]:
# why not use feature_dir and feature_file here?
# feature_path = f'{root_dir}features/{dataset_name}/pretrained_resnet18_features.npy'
# np.save(feature_path, all_feat)

In [ ]:
def get_save_features(model, feature_dir, feature_file, root_dir):
    dataset_names = ['CP', 'Allen', 'HPA']
    for dataset in dataset_names:
        dataset = configure_dataset(root_dir, dataset)
        train_dataloader = DataLoader(dataset, batch_size=256, shuffle=False)
        preprocess = weights.transforms()
        all_feat = []
        for images, label in tqdm(train_dataloader, total=len(train_dataloader)):
            cloned_images = images.clone()
            batch_feat = []
            for i in range(cloned_images.shape[1]):
                # Copy each channel three times 
                channel = cloned_images[:, i, :, :]
                channel = channel.unsqueeze(1)
                expanded = channel.expand(-1, 3, -1, -1).to(device)
        
                if type(model) == models.resnet.ResNet:
                    expanded = preprocess(expanded).to(device)
                    feat_temp = feature_extractor(expanded).cpu().detach().numpy()
                else: 
                    feat_temp = forward(expanded).cpu().detach().numpy()
                    
                batch_feat.append(feat_temp)
                
            batch_feat = np.concatenate(batch_feat, axis=1)
            all_feat.append(batch_feat)
       
        all_feat = np.concatenate(all_feat)
        all_feat = all_feat.squeeze(2).squeeze(2)
        feature_path = feature_path = f'{feature_dir}/{dataset}/{feature_file}'
        np.save(feature_path, all_feat)
